In [1]:
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
import numpy as np
import pandas as pd
import os
import cv2
import pickle
from flask import Flask, request, jsonify, render_template
from werkzeug.wrappers import  Request,Response
from werkzeug.serving import run_simple
import base64

In [2]:
detector = MTCNN()
embedder = FaceNet()
with open('svm_face_recognition_model.pkl', 'rb') as f:
    model = pickle.load(f)
class_names = ['Angelina Jolie', 'Brad Pitt', 'Denzel Washington', 'Hugh Jackman',
       'Jennifer Lawrence', 'Johnny Depp', 'Kate Winslet',
       'Leonardo DiCaprio', 'Megan Fox', 'Natalie Portman',
       'Nicole Kidman', 'Robert Downey Jr', 'Sandra Bullock',
       'Scarlett Johansson', 'Tom Cruise', 'Tom Hanks', 'Will Smith']

C:\Users\IAmAlight\anaconda3\envs\Ai_7\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
def get_face(img,box):
    x,y,w,h = box
    p = (x+w,y+h)
    face = img[y:y+h,x:x+w]
    face = cv2.resize(face,(160,160))
    return face,x,y,p

In [4]:
def get_embedding(face):
    face = cv2.resize(face,(160,160))
    face = face.astype('float32')
    face = np.expand_dims(face,axis=0)
    embedding = embedder.embeddings(face)
    return embedding[0]

In [5]:
def detect_recognize_faces(img):
    results = detector.detect_faces(img)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces = []
    boxes = []
    threshhold = 0.7
    for result in results:
        face,x,y,p = get_face(img,result['box'])
        embedding = get_embedding(face)
        faces.append(embedding)
        boxes.append((x,y,p))

    faces = np.array(faces)
    preds = model.predict_proba(faces)
    name_indices = np.argmax(preds,axis = 1)
    probas = preds[np.arange(len(name_indices)),name_indices]

    for i,(x,y,p) in enumerate(boxes):
        proba = probas[i]
        if proba>threshhold:
            name = class_names[name_indices[i]]
        else:
            name = "Unknown"

        cv2.rectangle(img,(x,y),p,[0,255,0],thickness=5)
        cv2.putText(img,f"{name}:{proba:.2f}",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.9,color = [0,255,0],thickness=4)
    return img


In [6]:
app = Flask(__name__)

@app.route('/',methods = ['GET','POST'])
def index():
    if request.method == 'POST':
        if 'image' not in request.files:
            return render_template('index.html',error = 'No image uploaded')

        file = request.files['image']
        img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_UNCHANGED)
        if img is None:
            return jsonify({'error': 'Invalid image'}), 400
            
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        result_img = detect_recognize_faces(img)
        result, buffer = cv2.imencode('.jpg', result_img)
        if result:
          img_str = base64.b64encode(buffer).decode('utf-8')
          return render_template('index.html',image=f'data:image/jpeg;base64,{img_str}')
        else:
          return jsonify({'error': 'Failed to encode image'}), 500
    return render_template('index.html')

In [ ]:
run_simple('localhost',8000,app)

 * Running on http://localhost:8000
Press CTRL+C to quit
127.0.0.1 - - [17/Sep/2024 21:16:23] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


127.0.0.1 - - [17/Sep/2024 21:16:34] "POST / HTTP/1.1" 200 -
